In [2]:
import tensorflow as tf
%matplotlib inline

In [3]:
# for class_name in classes:
#     class_path = os.path.join('assets', class_name)
#     images = os.listdir(class_path)
#     train_images, test_images = train_test_split(images, test_size= 0.2, random_state= 42)
#     train_class_dir = os.path.join(train_dir, class_name)
#     test_class_dir = os.path.join(test_dir, class_name)
#     os.makedirs(train_class_dir, exist_ok= True)
#     os.makedirs(test_class_dir, exist_ok= True)

#     for image in train_images:
#         src = os.path.join(class_path, image)
#         dst = os.path.join(train_class_dir, image)
#         shutil.copy(src, dst)

#     for image in test_images:
#         src = os.path.join(class_path, image)
#         dst = os.path.join(test_class_dir, image)
#         shutil.copy(src, dst)


In [3]:
train, validation = tf.keras.utils.image_dataset_from_directory(
    "assets",
    labels = "inferred",
    label_mode = "int",
    color_mode = 'grayscale',
    batch_size = 32,
    image_size = (28, 28),
    validation_split = 0.2,
    subset = "both",
    seed = 2,
    )

Found 6380 files belonging to 10 classes.
Using 5104 files for training.
Using 1276 files for validation.


## Creating a CNN

In [4]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(28, 28, 1)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=10, activation='softmax'))
cnn.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 64)               

In [5]:
cnn.compile(optimizer= 'adam', loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'])

In [6]:
cnn.fit(x= train, validation_data= validation, epochs=10)


Epoch 1/10


160/160 [==============================] - 5s 22ms/step - loss: 2.5452 - accuracy: 0.5264 - val_loss: 0.9155 - val_accuracy: 0.6904
Epoch 2/10
160/160 [==============================] - 2s 12ms/step - loss: 0.5453 - accuracy: 0.8166 - val_loss: 0.4403 - val_accuracy: 0.8605
Epoch 3/10
160/160 [==============================] - 2s 12ms/step - loss: 0.2179 - accuracy: 0.9271 - val_loss: 0.3287 - val_accuracy: 0.9067
Epoch 4/10
160/160 [==============================] - 2s 12ms/step - loss: 0.1136 - accuracy: 0.9653 - val_loss: 0.2302 - val_accuracy: 0.9271
Epoch 5/10
160/160 [==============================] - 2s 12ms/step - loss: 0.0612 - accuracy: 0.9833 - val_loss: 0.2120 - val_accuracy: 0.9373
Epoch 6/10
160/160 [==============================] - 2s 12ms/step - loss: 0.0740 - accuracy: 0.9773 - val_loss: 0.2018 - val_accuracy: 0.9436
Epoch 7/10
160/160 [==============================] - 2s 11ms/step - loss: 0.0515 - accuracy: 0.9837 - val_loss: 0.1632 - val_accuracy: 0.95

In [7]:
cnn.save("number_identifier")

INFO:tensorflow:Assets written to: number_identifier\assets


INFO:tensorflow:Assets written to: number_identifier\assets


In [8]:
from keras.preprocessing import image
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from keras.models import load_model
import cv2
import numpy as np
nums = []
cnn = load_model('number_identifier')
for i in range(81):
    path = "number_images/" + str(i) + ".jpg"
    test_image = image.load_img(path, target_size = (28, 28))
    test_image = image.img_to_array(test_image)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
    test_image = np.expand_dims(test_image, axis = 0)
    result = list(cnn.predict(test_image, verbose= 0)[0])
    nums.append(result.index(max(result)))
nums = np.reshape(nums, (9,9))
print(nums)

[[7 0 8 0 0 0 7 0 9]
 [6 3 0 2 0 9 0 8 4]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 5 0 0 0]
 [0 0 6 0 0 0 3 0 0]
 [0 8 0 6 0 8 0 1 0]
 [0 0 0 0 8 0 0 0 0]
 [8 4 0 8 0 2 0 7 1]
 [0 0 3 0 0 0 8 0 3]]
